# Train and Check accuracy on 20% test data

In [2]:
import sys
# sys.path.append("/content/drive/MyDrive/URL_Checker")  # this appends the parent directory to the sys.path list
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
import json
import tensorflow as tf
from nltk.util import ngrams
import itertools
from Utils import Evaluation
from Utils import DataProcessing, FeatureCreation, Model
N_for_NGram = 5
Sequence_length = 50
epochs = 5
n_classes = 2

2024-11-10 02:37:08.284674: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 02:37:08.711001: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 02:37:08.712907: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 02:37:09.848019: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from sklearn.model_selection import train_test_split
df1 = pd.read_csv("dataset/df_final.csv")
df2 = pd.read_csv("dataset/all_top_domains_merged.csv")
test_df = pd.read_csv("dataset/df_another_validation.csv")
url_dataframe = pd.concat([df1,df2], axis=0)
url_dataframe = url_dataframe.sample(frac=1).reset_index(drop=True)
url_dataframe = url_dataframe.drop_duplicates()
url_dataframe = url_dataframe.dropna().reset_index(drop=True)

In [4]:
train_df, val_df = train_test_split(url_dataframe, test_size=0.20, stratify=url_dataframe['type'])
# train_df, val_df = train_test_split(train_df, test_size=0.20, stratify=train_df['type'])

In [5]:
Batch_Size = 64
# Batch_Size = 128
max_tokens = 100000
# with strategy.scope():
train_ds = DataProcessing.process_train_data(train_df,Batch_Size)
Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower',
                                                    split="character",
                                                    ngrams=(N_for_NGram,),
                                                    output_mode='int',
                                                    max_tokens = max_tokens,
                                                    output_sequence_length=Sequence_length)
# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x,z: x[0])
Vectorize_Layer.adapt(train_text)
train_ds = train_ds.map(lambda x, z: DataProcessing.vectorize_text(x[0],x[1],z,Vectorize_Layer))
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

2024-11-10 02:37:54.455064: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 89309808 exceeds 10% of free system memory.
2024-11-10 02:37:54.570608: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 89309808 exceeds 10% of free system memory.
2024-11-10 02:37:54.599107: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [620207]
	 [[{{node Placeholder/_2}}]]
2024-11-10 02:38:10.088016: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 69805272 exceeds 10% of free system memory.
2024-11-10 02:38:13.783626: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 69805272 exceeds 10% of free system memory.


In [6]:
# with strategy.scope():
val_ds = DataProcessing.process_data(val_df,Batch_Size,Vectorize_Layer)
test_ds = DataProcessing.process_data(test_df,Batch_Size,Vectorize_Layer)

2024-11-10 02:38:51.250062: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 71901648 exceeds 10% of free system memory.


In [7]:
len(Vectorize_Layer.get_vocabulary())

100000

In [8]:
# with strategy.scope():
model = Model.create_model(Sequence_length, max_tokens, 18)
print(model.summary())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 18)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          2432        ['input_2[0][0]']                
                                                                                                  
 dropout_3 (Dropout)            (None, 128)          0           ['dense[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 256)          33024       ['dropout_3[0][0]']              
                                                                                            

In [9]:
history = model.fit(
    train_ds,
    epochs=epochs,
    batch_size=Batch_Size,
    validation_data=val_ds)

Epoch 1/5


2024-11-08 22:57:13.892197: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype resource
	 [[{{node Placeholder/_7}}]]


9691/9691 [==============================] - ETA: 0s - loss: 0.0767 - accuracy: 0.9733

2024-11-08 23:30:33.045258: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_17' with dtype resource
	 [[{{node Placeholder/_17}}]]
2024-11-08 23:30:33.045699: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [156335,18]
	 [[{{node Placeholder/_1}}]]


9691/9691 [==============================] - 2062s 212ms/step - loss: 0.0767 - accuracy: 0.9733 - val_loss: 0.0494 - val_accuracy: 0.9832
Epoch 2/5
9691/9691 [==============================] - 2035s 210ms/step - loss: 0.0384 - accuracy: 0.9871 - val_loss: 0.0446 - val_accuracy: 0.9851
Epoch 3/5
9691/9691 [==============================] - 2029s 209ms/step - loss: 0.0249 - accuracy: 0.9919 - val_loss: 0.0512 - val_accuracy: 0.9843
Epoch 4/5
9691/9691 [==============================] - 2014s 208ms/step - loss: 0.0169 - accuracy: 0.9944 - val_loss: 0.0578 - val_accuracy: 0.9844
Epoch 5/5
9691/9691 [==============================] - 2012s 208ms/step - loss: 0.0121 - accuracy: 0.9960 - val_loss: 0.0688 - val_accuracy: 0.9841


In [10]:
result = Evaluation.evaluate_model(model, test_ds, 0.5)

2024-11-09 01:46:32.014615: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype int64
	 [[{{node Placeholder/_20}}]]
2024-11-09 01:46:32.015122: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_17' with dtype resource
	 [[{{node Placeholder/_17}}]]


   2060/Unknown - 50s 24ms/step

7802/7802 [==============================] - 190s 24ms/step
Confusion Matrix:
[[  8521  98101]
 [ 46596 346099]]
Accuracy: 0.7102101470608851
Precision: 0.6457869524541044
Recall: 0.7102101470608851
F1 Score: 0.6729863240122619


Here it represents over 98.4% accuracy on validation data i.e. 20% of total data. and 71% accuracy for another dataset, which is not belongs to our dataset (it means our model is more generalized and will perform great for unseen data.)

In [11]:
links = ["itunes.apple.com/gb/album/whb-bonus-track-version/id358709625", "http://45.231.210.144:47317/Mozi.m", "www.google.com", "http://10.40.102.155:5009/get", "http://money.onion", "www.achnet.com", "https://colab.research.google.com/"]

In [12]:
Evaluation.make_prediction(model,Batch_Size, Vectorize_Layer, links, 0.5)

1/1 [==============================] - 0s 55ms/step


2024-11-09 01:49:51.689089: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype int64
	 [[{{node Placeholder/_20}}]]
2024-11-09 01:49:51.689550: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_17' with dtype resource
	 [[{{node Placeholder/_17}}]]
2024-11-09 01:49:51.789991: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_17' with dtype resource
	 [[{{node Placeholder/_17}}]]
2024-11-09

1/1 [==============================] - 0s 55ms/step


2024-11-09 01:49:51.892045: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_18' with dtype int64
	 [[{{node Placeholder/_18}}]]
2024-11-09 01:49:51.892475: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_19' with dtype string
	 [[{{node Placeholder/_19}}]]
2024-11-09 01:49:51.993459: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
20

1/1 [==============================] - 0s 58ms/step


2024-11-09 01:49:52.094561: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_19' with dtype string
	 [[{{node Placeholder/_19}}]]
2024-11-09 01:49:52.094983: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [1]
	 [[{{node Placeholder/_2}}]]
2024-11-09 01:49:52.194282: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [1]
	 [[{{node Placeholde

1/1 [==============================] - 0s 54ms/step


2024-11-09 01:49:52.298889: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_19' with dtype string
	 [[{{node Placeholder/_19}}]]
2024-11-09 01:49:52.299290: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_18' with dtype int64
	 [[{{node Placeholder/_18}}]]


['Safe', 'Malicious', 'Safe', 'Malicious', 'Malicious', 'Safe', 'Malicious']

In [3]:
df1 = pd.read_csv("dataset/df_final.csv")
df2 = pd.read_csv("dataset/all_top_domains_merged.csv")
df3 = pd.read_csv("dataset/df_another_validation.csv")
url_dataframe = pd.concat([df1,df2, df3], axis=0)
url_dataframe = url_dataframe.sample(frac=1).reset_index(drop=True)
url_dataframe = url_dataframe.drop_duplicates()
url_dataframe = url_dataframe.dropna().reset_index(drop=True)
# Batch_Size = 16 * strategy.num_replicas_in_sync
Batch_Size = 64
N_for_NGram = 5
Sequence_length = 50
epochs = 5
n_classes = 2
max_tokens = 100000
# with strategy.scope():
train_ds = DataProcessing.process_train_data(url_dataframe,Batch_Size)
Vectorize_Layer = tf.keras.layers.TextVectorization(standardize='lower',
                                                  split="character",
                                                  ngrams=(N_for_NGram,),
                                                  output_mode='int',
                                                  max_tokens = max_tokens,
                                                  output_sequence_length=Sequence_length)

# Make a text-only dataset (without labels), then call adapt
train_text = train_ds.map(lambda x,z: x[0])
Vectorize_Layer.adapt(train_text)
train_ds = train_ds.map(lambda x, z: DataProcessing.vectorize_text(x[0],x[1],z,Vectorize_Layer))
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

model = Model.create_model(Sequence_length, max_tokens, 18)
print(model.summary())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(
    train_ds,
    epochs=epochs,
    batch_size=Batch_Size)

2024-11-09 04:15:14.641614: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 120104784 exceeds 10% of free system memory.
2024-11-09 04:15:14.853219: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 120104784 exceeds 10% of free system memory.
2024-11-09 04:15:14.904349: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [834061]
	 [[{{node Placeholder/_2}}]]
2024-11-09 04:15:40.007919: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 89216856 exceeds 10% of free system memory.
2024-11-09 04:15:40.013035: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 29738952 exceeds 10% of free system memory.
2024-11-09 04:15:46.196574: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 89

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 18)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 128)          2432        ['input_2[0][0]']                
                                                                                                  
 dropout_3 (Dropout)            (None, 128)          0           ['dense[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 256)          33024       ['dropout_3[0][0]']              
                                                                                            

2024-11-09 04:15:47.859109: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [834061]
	 [[{{node Placeholder/_2}}]]


13033/13033 [==============================] - 2718s 208ms/step - loss: 0.1469 - accuracy: 0.9251
Epoch 2/5
13033/13033 [==============================] - 2685s 206ms/step - loss: 0.1073 - accuracy: 0.9405
Epoch 3/5
13033/13033 [==============================] - 2700s 207ms/step - loss: 0.0872 - accuracy: 0.9515
Epoch 4/5
13033/13033 [==============================] - 2695s 207ms/step - loss: 0.0707 - accuracy: 0.9631
Epoch 5/5
13033/13033 [==============================] - 2697s 207ms/step - loss: 0.0576 - accuracy: 0.9717


In [4]:
links = ["itunes.apple.com/gb/album/whb-bonus-track-version/id358709625","http://45.231.210.144:47317/Mozi.m","youtu.be", "www.google.com", "http://10.40.102.155:5009/get", "http://money.onion", "www.achnet.com", "https://colab.research.google.com/"]
Evaluation.make_prediction(model,1, Vectorize_Layer, links, 0.5)

2024-11-09 08:00:42.804410: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [1]
	 [[{{node Placeholder/_2}}]]
2024-11-09 08:00:42.804839: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype int64
	 [[{{node Placeholder/_20}}]]


1/1 [==============================] - 0s 51ms/step


2024-11-09 08:00:43.604414: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype int64
	 [[{{node Placeholder/_20}}]]
2024-11-09 08:00:43.604853: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_19' with dtype string
	 [[{{node Placeholder/_19}}]]
2024-11-09 08:00:43.702052: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_18' with dtype int64
	 [[{{node Placeholder/_18}}]]
2024-11-09 08:0

1/1 [==============================] - 0s 52ms/step


2024-11-09 08:00:43.905151: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [1]
	 [[{{node Placeholder/_2}}]]
2024-11-09 08:00:43.905849: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [1]
	 [[{{node Placeholder/_2}}]]
2024-11-09 08:00:44.013359: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [1,18]
	 [[{{n

1/1 [==============================] - 0s 54ms/step


2024-11-09 08:00:44.110451: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_20' with dtype int64
	 [[{{node Placeholder/_20}}]]
2024-11-09 08:00:44.110865: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_17' with dtype resource
	 [[{{node Placeholder/_17}}]]
2024-11-09 08:00:44.209979: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_17' with dtype resource
	 [[{{node Placeholder/_17}}]]
2024-11-09

['Safe',
 'Malicious',
 'Safe',
 'Safe',
 'Malicious',
 'Malicious',
 'Safe',
 'Malicious']

# Save the Model and Vocabulary of Vectorization Layer

In [5]:
model.save("Trained_Model/Model/Model.keras")

In [6]:
import json
vocab = Vectorize_Layer.get_vocabulary()
with open("Trained_Model/Vectorization_Layer/vocab.json", "w") as f:
    json.dump(vocab, f)

In [7]:
print(Batch_Size, Sequence_length, N_for_NGram, max_tokens)

64 50 5 100000


# Loading the Model and Vocabulary from Saved Model

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
import json
import tensorflow as tf
from nltk.util import ngrams
import itertools
from Utils import Evaluation
from Utils import DataProcessing, FeatureCreation, Model

2024-11-10 08:36:20.967978: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 08:36:23.571964: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-10 08:36:23.574814: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 08:36:28.432821: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
modell = tf.keras.models.load_model("Trained_Model/Model/Model.keras")

2024-11-10 08:36:46.541671: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51200000 exceeds 10% of free system memory.
2024-11-10 08:36:46.586931: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51200000 exceeds 10% of free system memory.
2024-11-10 08:36:46.597741: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51200000 exceeds 10% of free system memory.
2024-11-10 08:36:48.696278: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51200000 exceeds 10% of free system memory.
2024-11-10 08:36:48.755745: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 51200000 exceeds 10% of free system memory.


In [3]:
# Batch_Size = 16 * strategy.num_replicas_in_sync
Batch_Size = 64
max_tokens = 100000
N_for_NGram = 5
Sequence_length = 50
with open("Trained_Model/Vectorization_Layer/vocab.json", "r") as f:
    vocab = json.load(f)
VL = tf.keras.layers.TextVectorization(standardize='lower',
                                                   split="character",
                                                   ngrams=(N_for_NGram,),
                                                   output_mode='int',
                                                   output_sequence_length=Sequence_length,
                                                   vocabulary=vocab)

In [4]:
links = ["itunes.apple.com/gb/album/whb-bonus-track-version/id358709625","http://45.231.210.144:47317/Mozi.m","youtu.be",
          "www.google.com", "http://10.40.102.155:5009/get"]
Evaluation.make_prediction(modell,Batch_Size, VL, links, 0.5)

2024-11-10 08:36:52.050142: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_19' with dtype string
	 [[{{node Placeholder/_19}}]]
2024-11-10 08:36:52.050987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_18' with dtype int64
	 [[{{node Placeholder/_18}}]]


1/1 [==============================] - 0s 64ms/step


2024-11-10 08:36:53.469437: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_19' with dtype string
	 [[{{node Placeholder/_19}}]]
2024-11-10 08:36:53.470169: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype int64 and shape [1]
	 [[{{node Placeholder/_2}}]]
2024-11-10 08:36:53.646203: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_17' with dtype resource
	 [[{{node Placeholder/_17}}]]


1/1 [==============================] - 0s 69ms/step


2024-11-10 08:36:53.813958: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_17' with dtype resource
	 [[{{node Placeholder/_17}}]]
2024-11-10 08:36:53.814459: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_19' with dtype string
	 [[{{node Placeholder/_19}}]]
2024-11-10 08:36:54.006123: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_19' with dtype string
	 [[{{node Placeholder/_19}}]]
2024-11-10 

1/1 [==============================] - 0s 72ms/step


['Safe', 'Malicious', 'Safe', 'Safe', 'Malicious']